In [ ]:
!pip install langchain langchain-openai langchain-community huggingface_hub PyPDF2 langchain-huggingface faiss-cpu langchain-groq flask-ngrok

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-huggingface to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-huggingface to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-groq to determine which version is compatible with other requirements

In [ ]:
!pip uninstall -y langchain
!pip install langchain==0.1.20


Found existing installation: langchain 0.1.20
Uninstalling langchain-0.1.20:
  Successfully uninstalled langchain-0.1.20
  Using cached langchain-0.1.20-py3-none-any.whl.metadata (13 kB)
Using cached langchain-0.1.20-py3-none-any.whl (1.0 MB)


In [ ]:
!pip install dotenv

In [ ]:
from PyPDF2 import PdfReader
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.memory.buffer import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from huggingface_hub import login
from langchain_groq import ChatGroq
import os
from flask import Flask, render_template_string
from flask_ngrok import run_with_ngrok
from google.colab import userdata

In [ ]:
login(os.getenv("HUGGINGFACEHUB_API_TOKEN"))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving NEP_Final_English_0.pdf to NEP_Final_English_0.pdf


In [ ]:
import os
pdf_dir = "/content"
pdf_files = [os.path.join(pdf_dir, f) for f in os.listdir(pdf_dir) if f.endswith(".pdf")]

print("Selected PDF files:", pdf_files)


Selected PDF files: ['/content/NEP_Final_English_0.pdf']


In [ ]:
def get_pdf_text(pdf_docs):
  '''
  loop over all the pdf files and concatenate
  the content in a single string
  '''
  text = ""
  for pdf in pdf_docs:
    pdf_reader = PdfReader(pdf)
    for page in pdf_reader.pages:
      page_text = page.extract_text()
      if page_text:
        text += page_text
  return text

In [ ]:
raw_text = get_pdf_text(pdf_files)
print("✅ Extracted text length:", len(raw_text))

✅ Extracted text length: 300298


In [ ]:
def get_text_chunks(raw_text):
  '''
  takes a single string of text and returns a list
  of text strings that can be fed to vector database
  '''
  text_splitter = CharacterTextSplitter(
      separator="\n",
      chunk_size=1000,
      chunk_overlap=200,
      length_function=len
  )
  chunks = text_splitter.split_text(raw_text)
  return chunks


In [ ]:
text_chunks = get_text_chunks(raw_text)
print("✅ Number of text chunks:", len(text_chunks))

✅ Number of text chunks: 364


In [ ]:
def get_vectorstore(text_chunks):
  '''
  creat a FAISS vectorstore from the text chunks using embeddings
  '''
  embeddings = HuggingFaceEmbeddings(
      model_name="hkunlp/instructor-xl",
      model_kwargs={"device": "cuda"}
  )
  vectorstore = FAISS.from_texts(
      texts=text_chunks,
      embedding=embeddings
  )
  return vectorstore


In [ ]:
vectorstore = get_vectorstore(text_chunks)
print("✅ vectorstore created")

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

2_Dense/pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

✅ vectorstore created


In [ ]:
def get_conversation_chain(vectorstore):
    '''
    Creates a conversational retrieval chain using OpenAI's mode
    '''
   #llm = ChatOpenAI(
     # model_name= "gpt-4o-mini",
     # temperature=0.7,
     # request_timeout=30
    llm = ChatGroq(
         model_name="llama-3.3-70b-versatile",
         temperature=0.7,
         request_timeout=30,
         api_key=userdata.get('GROQ_API_KEY')
    )
    memory = ConversationBufferMemory(
        memory_key='chat_history',
        return_messages=True
    )
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain


In [ ]:
conversation = get_conversation_chain(vectorstore)

In [ ]:
def chat_with_pdf(question):
    response = conversation({'question': question})
    for i, msg in enumerate(response['chat_history']):
      sender = "User" if i % 2 == 0 else "BOT"
      print(f"{sender}: {msg.content}")

In [ ]:
chat_with_pdf("What is the introduction of Nation Education Policy 2020")

/usr/local/lib/python3.12/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


User: What is the introduction of Nation Education Policy 2020
BOT: The introduction of the National Education Policy 2020 is as follows:

"Education is fundamental for achieving full human potential, developing an equitable and just society, and promoting national development. Providing universal access to quality education is the aim of the National Education Policy 2020, which envisions an education system that is second to none, with equitable access to the highest-quality education for all learners, regardless of social or economic background."


In [ ]:
chat_with_pdf("What are the Other key areas of focus?")

User: What is the introduction of Nation Education Policy 2020
BOT: The introduction of the National Education Policy 2020 is as follows:

"Education is fundamental for achieving full human potential, developing an equitable and just society, and promoting national development. Providing universal access to quality education is the aim of the National Education Policy 2020, which envisions an education system that is second to none, with equitable access to the highest-quality education for all learners, regardless of social or economic background."
User: What are the Other key areas of focus?
BOT: The other key areas of focus in the National Education Policy 2020 are:

1. Professional Education (Section 20)
2. Adult Education and Life Long Learning (Section 21)
3. Promotion of Indian Languages, Arts and Culture (Section 22)
4. Technology Use and Integration (Section 23)
5. Online and Digital Education: Ensuring Equitable Use of Technology (Section 24)

These areas are listed under Par

In [ ]:
chat_with_pdf("What is the role of Technology use and Integration ")

User: What is the introduction of Nation Education Policy 2020
BOT: The introduction of the National Education Policy 2020 is as follows:

"Education is fundamental for achieving full human potential, developing an equitable and just society, and promoting national development. Providing universal access to quality education is the aim of the National Education Policy 2020, which envisions an education system that is second to none, with equitable access to the highest-quality education for all learners, regardless of social or economic background."
User: What are the Other key areas of focus?
BOT: The other key areas of focus in the National Education Policy 2020 are:

1. Professional Education (Section 20)
2. Adult Education and Life Long Learning (Section 21)
3. Promotion of Indian Languages, Arts and Culture (Section 22)
4. Technology Use and Integration (Section 23)
5. Online and Digital Education: Ensuring Equitable Use of Technology (Section 24)

These areas are listed under Par

UI/UX

In [ ]:
import gradio as gr

# Reuse the same conversation object created in the notebook
def query_bot(question, chat_history=[]):
    response = conversation({'question': question})
    answer = response['answer']
    chat_history.append((question, answer))
    return chat_history, ""

# Sample queries
sample_queries = [
    "Tell me about Education policies",
    "Explain the latest education policy",
    "What are the government schemes for students?",
    "Describe the employment programs available",
]

with gr.Blocks() as demo:
    gr.Markdown("<h1 style='text-align: center;'>🛡 Government Policy Chatbot</h1>")

    chatbot = gr.Chatbot(label="Policy Assistant")
    query_input = gr.Textbox(label="Ask a Question", placeholder="Type your question here...", lines=1)
    submit_button = gr.Button("Submit")

    with gr.Row():
        buttons = []
        for q in sample_queries:
            btn = gr.Button(q)
            buttons.append(btn)

    submit_button.click(fn=query_bot, inputs=[query_input, chatbot], outputs=[chatbot, query_input])

    for btn in buttons:
        btn.click(lambda q=btn.value: query_bot(q, []), outputs=[chatbot, query_input])

demo.launch()


/tmp/ipython-input-2957249934.py:21: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Policy Assistant")
/tmp/ipython-input-2957249934.py:21: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(label="Policy Assistant")


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c601b45b0bac1dcb2c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
